In [14]:
import pandas as pd
import numpy as np
import xarray as xr

import os
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
from pyaldata import *

In [157]:
# Load data
data_dir = "./data/"
fname = os.path.join(data_dir, "RS_ind_1.mat")

In [158]:
# load TrialData .mat file into a DataFrame
df = mat2dataframe(fname, shift_idx_fields=True)

In [159]:
# Preprocessing

# combine time bins into longer ones
td = combine_time_bins(df, 3)
print(df.head().bin_size)
print()
print(td.head().bin_size)

0    0.01
1    0.01
2    0.01
3    0.01
4    0.01
Name: bin_size, dtype: float64

0    0.03
1    0.03
2    0.03
3    0.03
4    0.03
Name: bin_size, dtype: float64


In [160]:
print("original: ", df.M1_spikes[0].shape)
print("combined: ", td.M1_spikes[0].shape)

original:  (86, 67)
combined:  (28, 67)


In [161]:
# Remove low-firing neurons
td = remove_low_firing_neurons(td, "M1_spikes",  3)
td = remove_low_firing_neurons(td, "PMd_spikes", 3)

print("original: ", df.M1_spikes[0].shape)
print("removed:  ", td.M1_spikes[0].shape)

original:  (86, 67)
removed:   (28, 59)


In [162]:
# Smooting the signal

td = smooth_signals(td, ["M1_spikes", "PMd_spikes"], std=0.05)



In [163]:
# Combine M1 and PMd
td = merge_signals(td, ["M1_spikes", "PMd_spikes"], "both_spikes")

In [164]:
print(td.M1_spikes[1].shape, td.PMd_spikes[1].shape)
print(td.both_spikes[1].shape)


(32, 59) (32, 33)
(32, 92)


In [165]:
N = td.shape[0]

# Smallest length of a successful trial
#min_len = 20

for i in range(N):
    
    
    # get position
    pos = td.pos[i]

    
    # get velocities for this trial
    vel = td.vel[i]
   
    
    # get M1 spikes
    M1 = td.M1_spikes[i]
    
    # get PMd_spikes
    PMd = td.PMd_spikes[i]
    
    # get both
    both_spikes = td.both_spikes[i]
    
    
    if (i == 0):
        grouped_pos = pos
        grouped_vel = vel

        grouped_M1 = M1
        grouped_PMd = PMd
        grouped_both_spikes = both_spikes
    else:
        grouped_pos = np.append(grouped_pos, pos, axis=0)
        grouped_vel = np.append(grouped_vel, vel, axis=0)
        grouped_M1 = np.append(grouped_M1, M1, axis=0)
        grouped_PMd = np.append(grouped_PMd, PMd, axis=0)
        grouped_both_spikes = np.append(grouped_both_spikes, both_spikes, axis=0);
 
print(grouped_pos.shape)    
print(grouped_vel.shape)
print(grouped_M1.shape)
print(grouped_PMd.shape)
print(grouped_both_spikes.shape)
   
    

(2387, 2)
(2387, 2)
(2387, 59)
(2387, 33)
(2387, 92)


In [166]:
# Save the data in right format
import pickle

data_folder='data/' #FOLDER YOU WANT TO SAVE THE DATA TO

with open(data_folder+'RS_ind_1.pickle','wb') as f:
    pickle.dump([grouped_M1, grouped_PMd, grouped_both_spikes, grouped_pos, grouped_vel],f)